# Ontologies to use
- **Post-Subreddit:** A post belongs to a specific subreddit.
- **Post-Author:** A post is created by an author.
- **Post-Topics:** A post is related to one or more topics based on the topic modeling results.
- **Post-Comments:** A post has a set of comments.
- **Post-Keywords:** A post is associated with specific keywords derived from the topic modeling.

## Loading libraries to be used

In [3]:
# Download NLTK resources 
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
!pip install pyvis



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import re
import json
import rdflib
import gensim
import pandas as pd
import networkx as nx
from gensim import corpora
from pymongo import MongoClient
from nltk.corpus import stopwords
import plotly.graph_objects as go
from pyvis.network import Network
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [5]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\97156\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\97156\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\97156\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\97156\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Reading data from *MongoDB*

- Connect to MongoDB
- Read the data from the collection
- Convert the data into a pandas DataFrame
- Print the first few rows of the DataFrame

In [6]:
client = MongoClient("mongodb://localhost:27017/")
db = client['DataTails']
collection = db['Data']  
data_cursor = collection.find({})
DF = pd.DataFrame(list(data_cursor))
print(DF.head())

                        _id  type  \
0  66e965e698330736e0d693d5  None   
1  66e965e698330736e0d693d6  None   
2  66e965e698330736e0d693d7  None   
3  66e965e698330736e0d693d8  None   
4  66e965e698330736e0d693d9  None   

                                           postTitle postDesc  \
0  Adults(especially those over 30), how young do...      NaN   
1  What is a thing that your parents consider nor...      NaN   
2                 What is a smell that comforts you?      NaN   
3  When in history was it the best time to be a w...      NaN   
4    What's the worst way someone broke up with you?      NaN   

              postTime            authorName noOfUpvotes isNSFW  \
0  2024-08-06 01:02:35  Excellent-Studio7257        4068  False   
1  2024-08-06 01:47:22        Bigbumoffhappy        2073  False   
2  2024-08-05 22:21:53         bloomsmittenn        2188  False   
3  2024-08-06 03:32:59   More_food_please_77         778  False   
4  2024-08-05 21:01:39    ImpressiveWrap7363       

## Preprocessing the Data

- **Stopword Removal & Lemmatization:** 
    - Preprocessing() uses NLTK to remove stopwords and lemmatize the text.
- **Handling Missing Data:**
    - Missing postDesc fields are filled with an empty string.
    - Missing noOfUpvotes is filled with 0.
- **Datetime Conversion:** 
    - postTime is converted to a datetime object, and any errors are coerced.
- **Handling Comments:** 
    - Comments are converted from list format to a string of concatenated comments.
- **Final Clean Text:** 
    - Both postTitle and postDesc are cleaned using regular expressions and tokenization, and then passed through the NLTK-based text preprocessing function.

In [7]:
lemmatizer = WordNetLemmatizer()
StopWords = set(stopwords.words('english'))
custom_stopwords = StopWords | {"make", "thing", "know", "get", "want", "like", "would", "could", "you", "say","also","aita","com","www","made","ago","day","000"}
def Preprocessing(text):
    text = re.sub(r'\W', ' ', text)  # Remove non-alphanumeric characters
    text = re.sub(r'\b\w{1,2}\b', '', text)  # Remove short words
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in custom_stopwords and len(word) > 2]
    return tokens

Cols = ['subReddit', 'postTitle', 'postDesc', 'postTime', 'authorName', 'noOfUpvotes', 'comments', 'noOfComments', 'postUrl','imageUrl','isNSFW']
DF = DF[Cols]
print(DF.isnull().sum())


DF['subReddit'] = DF['subReddit'].fillna('Unknown_SubReddit')
DF['authorName'] = DF['authorName'].fillna('Unknown_Author')
DF['postTitle'] = DF['postTitle'].fillna('Untitled')
DF['postUrl'] = DF['postUrl'].fillna('http://example.com/NOPOST.png')
DF['imageUrl'] = DF['imageUrl'].fillna('http://example.com/NOImage.png')
DF['isNSFW'] = DF['isNSFW'].fillna(False)
DF['postDesc'].fillna('', inplace=True)
DF['noOfUpvotes'].fillna(0, inplace=True)
DF['noOfComments'].fillna(0, inplace=True)
DF['postTime'] = pd.to_datetime(DF['postTime'], errors='coerce')
DF['comments'] = DF['comments'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x))
DF['postTitle'] = DF['postTitle'].apply(lambda x: Preprocessing(str(x)))
DF['postDesc'] = DF['postDesc'].apply(lambda x: Preprocessing(str(x)))
DF['isNSFW'] = DF['isNSFW'].astype(bool)
print(DF.head())
print(DF.isnull().sum())

subReddit           2
postTitle           0
postDesc        15607
postTime            0
authorName        587
noOfUpvotes         0
comments            0
noOfComments        2
postUrl             2
imageUrl            3
isNSFW              1
dtype: int64
   subReddit                                          postTitle postDesc  \
0  AskReddit                   [adult, especially, young, seem]       []   
1  AskReddit  [parent, consider, normal, consider, normal, a...       []   
2  AskReddit                                   [smell, comfort]       []   
3  AskReddit                       [history, best, time, woman]       []   
4  AskReddit                       [worst, way, someone, broke]       []   

             postTime            authorName noOfUpvotes  \
0 2024-08-06 01:02:35  Excellent-Studio7257        4068   
1 2024-08-06 01:47:22        Bigbumoffhappy        2073   
2 2024-08-05 22:21:53         bloomsmittenn        2188   
3 2024-08-06 03:32:59   More_food_please_77         

In [8]:
import rdflib
from rdflib import URIRef, Literal
from collections import defaultdict
import gensim
from gensim import corpora
import json

# RDF graph initialization
g = rdflib.Graph()
SIOC = rdflib.Namespace("http://rdfs.org/sioc/ns#")
DCMI = rdflib.Namespace("http://purl.org/dc/elements/1.1/")
FOAF = rdflib.Namespace("http://xmlns.com/foaf/0.1/")
REDDIT = rdflib.Namespace("http://reddit.com/ns#")  # Custom namespace for Reddit-specific relationships
g.bind("sioc", SIOC)
g.bind("dc", DCMI)
g.bind("foaf", FOAF)
g.bind("reddit", REDDIT)


# Function to add post data to RDF graph
def add_post_to_graph(row, index, topic_uri, subreddit_type_uri, author_type_uri, post_type_uri, comment_type_uri):
    post_uri = URIRef(f"http://reddit.com/post{index}")
    subreddit_uri = URIRef(f"http://reddit.com/subreddit/{row['subReddit']}")
    author_uri = URIRef(f"http://reddit.com/user/{row['authorName']}")
    
    # Add post properties and link to topic
    g.add((post_uri, rdflib.RDF.type, SIOC.Post))
    g.add((post_uri, DCMI.title, Literal(' '.join(row['postTitle']))))
    g.add((post_uri, DCMI.description, Literal(' '.join(row['postDesc']))))
    g.add((post_uri, DCMI.date, Literal(row['postTime'])))
    g.add((post_uri, SIOC.num_replies, Literal(row['noOfUpvotes'])))
    g.add((post_uri, SIOC.link, URIRef(row['postUrl'])))
    g.add((post_uri, SIOC.NSFW, Literal(row['isNSFW'])))
    g.add((post_uri, SIOC.has_type, post_type_uri))
    g.add((post_uri, SIOC.topic, topic_uri))

    # Create the relationships based on the list of relationship types
    # CreatedBy: Indicates that a post is created by an author.
    g.add((post_uri, REDDIT.CreatedBy, author_uri))

    # BelongsTo: Links a post to a subreddit.
    g.add((post_uri, REDDIT.BelongsTo, subreddit_uri))

    # PartOfPost: Indicates that a comment is part of a post (the comment belongs to the post).
    comment_uri = URIRef(f"http://reddit.com/comment/{index}")
    g.add((comment_uri, REDDIT.PartOfPost, post_uri))

    # HasType: Links an entity (post, comment) to its specific type (text post, link post, etc.).
    g.add((post_uri, REDDIT.HasType, post_type_uri))
    g.add((comment_uri, REDDIT.HasType, comment_type_uri))

    # TaggedIn: Indicates that a post or comment is tagged with a specific topic or keyword.
    for tag in row.get('tags', []):  # Assuming tags are available as a list
        tag_uri = URIRef(f"http://reddit.com/tag/{tag}")
        g.add((post_uri, REDDIT.TaggedIn, tag_uri))
        g.add((comment_uri, REDDIT.TaggedIn, tag_uri))

    # UpvotedBy: Indicates that a user has upvoted a post or comment.
    if row.get('upvotedBy'):
        for upvoter in row['upvotedBy']:
            upvoter_uri = URIRef(f"http://reddit.com/user/{upvoter}")
            g.add((post_uri, REDDIT.UpvotedBy, upvoter_uri))
            g.add((comment_uri, REDDIT.UpvotedBy, upvoter_uri))

    # DownvotedBy: Indicates that a user has downvoted a post or comment.
    if row.get('downvotedBy'):
        for downvoter in row['downvotedBy']:
            downvoter_uri = URIRef(f"http://reddit.com/user/{downvoter}")
            g.add((post_uri, REDDIT.DownvotedBy, downvoter_uri))
            g.add((comment_uri, REDDIT.DownvotedBy, downvoter_uri))

    # Moderates: Indicates that a user is a moderator of a subreddit.
    if row.get('moderators'):
        for moderator in row['moderators']:
            moderator_uri = URIRef(f"http://reddit.com/user/{moderator}")
            g.add((subreddit_uri, REDDIT.Moderates, moderator_uri))

    # HasFlair: Links a post or comment to a specific flair (e.g., tags like "Important", "Question", etc.).
    if row.get('flair'):
        flair_uri = URIRef(f"http://reddit.com/flair/{row['flair']}")
        g.add((post_uri, REDDIT.HasFlair, flair_uri))
        g.add((comment_uri, REDDIT.HasFlair, flair_uri))

    # Link post to subreddit and type
    g.add((subreddit_uri, rdflib.RDF.type, SIOC.Container))
    g.add((subreddit_uri, SIOC.has_post, post_uri))
    g.add((subreddit_uri, SIOC.has_type, subreddit_type_uri))
    g.add((post_uri, SIOC.Container, subreddit_uri))

    # Link to author and author type
    g.add((author_uri, rdflib.RDF.type, FOAF.Person))
    g.add((author_uri, FOAF.name, Literal(row['authorName'])))
    g.add((post_uri, SIOC.has_creator, author_uri))
    g.add((author_uri, SIOC.has_type, author_type_uri))

    # Add comments and link to comment type
    g.add((comment_uri, rdflib.RDF.type, SIOC.Comment))
    g.add((comment_uri, DCMI.title, Literal(row['comments'])))
    g.add((post_uri, SIOC.has_reply, comment_uri))
    g.add((comment_uri, SIOC.has_type, comment_type_uri))

# Process each subreddit and create LDA models
GroupedData = DF.groupby('subReddit')
all_topics = defaultdict(set)

for subreddit, group in GroupedData:
    group['Combined'] = group['postTitle'] + group['postDesc']
    group['Tokens'] = group['Combined'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x if word not in custom_stopwords])

    # Create dictionary and corpus
    dictionary = corpora.Dictionary(group['Tokens'])
    corpus = [dictionary.doc2bow(text) for text in group['Tokens']]
    LDA = gensim.models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15, iterations=100, random_state=42)
    
    for idx, topic in LDA.print_topics(num_words=5):
        topic_uri = URIRef(f"http://reddit.com/topic/{subreddit}_{idx}")
        topic_words = topic.split(" + ")
        unique_words = {word.split("*")[1].strip('"') for word in topic_words}
        all_topics[subreddit].update(unique_words)

        # Define URIs for types
        subreddit_type_uri = URIRef(f"http://reddit.com/type/subreddit/{subreddit}")
        post_type_uri = URIRef(f"http://reddit.com/type/post/{subreddit}")
        comment_type_uri = URIRef(f"http://reddit.com/type/comment/{subreddit}")
        author_type_uri = URIRef(f"http://reddit.com/type/author/{subreddit}")

        # Link subreddit to topic
        subreddit_uri = URIRef(f"http://reddit.com/subreddit/{subreddit}")
        g.add((subreddit_uri, SIOC.has_topic, topic_uri))
        g.add((topic_uri, rdflib.RDF.type, SIOC.Topic))

        # Iterate through each row in the group
        for index, row in group.iterrows():
            add_post_to_graph(row, index, topic_uri, subreddit_type_uri, author_type_uri, post_type_uri, comment_type_uri)

# Save graph
g.serialize('D:/FYP/Github/data-tails/Backend/Ontologies/KG.ttl', format='turtle')
json_ld = g.serialize(format='json-ld', indent=4)
with open("D:/FYP/Github/data-tails/Backend/Ontologies/KG.json", "w", encoding="utf-8") as f:
    f.write(json_ld)

### Converting *KG.json, KG.ttl* to format of D3 input file to view graph

In [9]:
import json
import rdflib
# from rdflib.namespace import RDF, SIOC, REDDIT

# Initialize D3-compatible JSON structure
d3_data = {
    "nodes": [],
    "links": []
}

def parse_label(url):
    if "subreddit" in url:
        # Extract subreddit name after /subreddit/
        name = url.split("/subreddit/")[1]
        return f"Subreddit({name})"
    elif "topic" in url:
        # Extract topic name after /topic/
        name = url.split("/topic/")[1]
        return f"Topic({name})"
    elif "post" in url:
        # Extract post ID after /post
        name = url.split("/reddit.com/")[1]
        return f"Post({name})"
    elif "user" in url:
        # Extract username after /user/
        name = url.split("/user/")[1]
        return f"Author({name})"
    elif "comment" in url:
        # Extract comment ID after /comment/
        name = url.split("/comment/")[1]
        return f"Comment({name})"
    elif "tag" in url:
        # Extract comment ID after /comment/
        name = url.split("/tag/")[1]
        return f"Tag({name})"
    elif "upvoter" in url:
        # Extract comment ID after /comment/
        name = url.split("/upvoter/")[1]
        return f"UpVote({name})"
    elif "downvoter" in url:
        # Extract comment ID after /comment/
        name = url.split("/downvoter/")[1]
        return f"DownVote({name})"
    return url  # In case no specific type is found, return the URL itself

# Create a mapping for nodes to avoid duplicates
node_map = {}

# Function to add a node if it doesn't already exist
def add_node(label, node_type, group, description=""):
    if label not in node_map:
        node_id = f"{node_type}_{len(d3_data['nodes']) + 1}"  # Create a unique node ID
        node_map[label] = node_id
        d3_data["nodes"].append({
            "id": node_id,
            "label": label,
            "type": node_type,
            "group": group  # Group for categorization
        })
    return node_map[label]

# Populate nodes and links based on the RDF graph structure
for subreddit in g.subjects(rdflib.RDF.type, SIOC.Container):
    subreddit_label = str(subreddit)
    subreddit_label= parse_label(subreddit_label)
    subreddit_id = add_node(subreddit_label, "Subreddit", group=0, description="Community for specific topics")

    # For each post in the subreddit
    for post in g.objects(subreddit, SIOC.has_post):
        post_label = str(post)
        post_label= parse_label(post_label)
        post_id = add_node(post_label, "Post", group=1, description="Individual posts in the subreddit")

        # Create a link from subreddit to post
        d3_data["links"].append({
            "source": subreddit_id,
            "target": post_id,
            "type": "Contains",
            "weight": 1  # Link weight can be adjusted based on relevance or count
        })

        # Add authors
        author = g.value(post, SIOC.has_creator)
        if author:
            author_label = str(author)
            author_label= parse_label(author_label)
            author_id = add_node(author_label, "Author", group=2, description="User who created the post")

            # Create a link from post to author
            d3_data["links"].append({
                "source": post_id,
                "target": author_id,
                "type": "CreatedBy",
                "weight": 1
            })

        # For each comment on the post
        for comment in g.objects(post, SIOC.has_reply):
            comment_label = str(comment)
            comment_label= parse_label(comment_label)
            comment_id = add_node(comment_label, "Comment", group=3, description="User comments on the post")

            # Create a link from post to comment
            d3_data["links"].append({
                "source": post_id,
                "target": comment_id,
                "type": "HasReply",
                "weight": 1
            })

            # Link comment to author if available
            if author:
                d3_data["links"].append({
                    "source": author_id,
                    "target": comment_id,
                    "type": "CommentedBy",
                    "weight": 1
                })

            # Add tags to comment if available
            for tag in g.objects(post, REDDIT.TaggedIn):
                tag_label = str(tag)
                tag_label= parse_label(tag_label)
                tag_id = add_node(tag_label, "Tag", group=4, description="Tags related to the post or comment")
                d3_data["links"].append({
                    "source": comment_id,
                    "target": tag_id,
                    "type": "TaggedWith",
                    "weight": 1
                })

        # Add topics to post
        for topic in g.objects(post, SIOC.topic):
            topic_label = str(topic)
            topic_label= parse_label(topic_label)
            topic_id = add_node(topic_label, "Topic", group=5, description="Topic related to the post")

            # Create a link from post to topic
            d3_data["links"].append({
                "source": post_id,
                "target": topic_id,
                "type": "RelatedTo",
                "weight": 1
            })

        # Add votes (upvotes and downvotes)
        for upvoter in g.objects(post, REDDIT.UpvotedBy):
            upvoter_label = str(upvoter)
            upvoter_label= parse_label(upvoter_label)
            upvoter_id = add_node(upvoter_label, "Upvoter", group=6, description="User who upvoted the post")
            d3_data["links"].append({
                "source": post_id,
                "target": upvoter_id,
                "type": "UpvotedBy",
                "weight": 1
            })

        for downvoter in g.objects(post, REDDIT.DownvotedBy):
            downvoter_label = str(downvoter)
            downvoter_label= parse_label(downvoter_label)
            downvoter_id = add_node(downvoter_label, "Downvoter", group=7, description="User who downvoted the post")
            d3_data["links"].append({
                "source": post_id,
                "target": downvoter_id,
                "type": "DownvotedBy",
                "weight": 1
            })

# Write the D3-compatible JSON data to a file
with open("D:/FYP/Github/data-tails/Backend/Ontologies/D3KG.json", "w", encoding="utf-8") as f:
    json.dump(d3_data, f, indent=4, ensure_ascii=False)



## BFS

In [10]:
import json
import rdflib
from collections import deque

# Initialize D3-compatible JSON structure
d3_data = {
    "nodes": [],
    "links": []
}

# Create a mapping for nodes to avoid duplicates
node_map = {}

# Function to add a node if it doesn't already exist
def add_node(label, node_type, group, description):
    if label not in node_map:
        node_id = f"{node_type}_{len(d3_data['nodes']) + 1}"  # Create a unique node ID
        node_map[label] = node_id
        d3_data["nodes"].append({
            "id": node_id,
            "label": label,
            "type": node_type,
            "group": group,
            "description": description
        })
    return node_map[label]

# Function to perform BFS and retrieve a subgraph
def bfs(start_node_id, max_depth):
    visited = set()
    queue = deque([(start_node_id, 0)])  # (node_id, depth)
    nodes_to_display = set([start_node_id])
    links_to_display = []

    # Mark the starting node to highlight it
    start_node_found = False

    while queue:
        current_node_id, current_depth = queue.popleft()
        if current_depth < max_depth:
            # Add links of current node to the links_to_display
            for link in d3_data["links"]:
                if link["source"] == current_node_id and link["target"] not in visited:
                    links_to_display.append(link)
                    queue.append((link["target"], current_depth + 1))
                elif link["target"] == current_node_id and link["source"] not in visited:
                    links_to_display.append(link)
                    queue.append((link["source"], current_depth + 1))

        visited.add(current_node_id)

        # Highlight the start node
        if current_node_id == start_node_id and not start_node_found:
            start_node_found = True
            for node in d3_data["nodes"]:
                if node["id"] == start_node_id:
                    node["highlighted"] = "Head"

    # Filter the nodes that are part of the display
    for link in links_to_display:
        nodes_to_display.add(link["source"])
        nodes_to_display.add(link["target"])

    # Filter out the nodes and links that should be displayed
    filtered_nodes = [node for node in d3_data["nodes"] if node["id"] in nodes_to_display]
    filtered_links = [link for link in links_to_display]

    return filtered_nodes, filtered_links

# Load the RDF data from the D3KG.json file
with open("D3KG.json", "r", encoding="utf-8") as f:
    d3_data = json.load(f)

# Example of how to call BFS with a starting node and max depth
start_node = "Author_3"
max_depth = 2
filtered_nodes, filtered_links = bfs(start_node, max_depth)

# Write the filtered subgraph to a file
with open("BFS.json", "w", encoding="utf-8") as f:
    json.dump({"nodes": filtered_nodes, "links": filtered_links}, f, indent=4, ensure_ascii=False)


## GRAPH RAG